In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from API_Keys import Google_API_Key

In [2]:
City_Data_df = pd.read_csv("Output_Files/Cities.csv")
City_Data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mantua,45.17,10.78,84.99,48,79,3.00,IT,1590244146
1,1,mataura,-46.19,168.86,46.02,89,100,2.62,NZ,1590244146
2,2,tocopilla,-22.09,-70.20,70.02,42,5,1.92,CL,1590244146
3,3,qaanaaq,77.48,-69.36,31.37,90,0,3.18,GL,1590244146
4,4,bilibino,68.05,166.44,26.24,92,82,2.33,RU,1590244147


In [3]:
gmaps.configure(api_key=Google_API_Key)

In [4]:
locations = City_Data_df[["Lat", "Lng"]]
humidity = City_Data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
narrowed_city_df = City_Data_df.loc[(City_Data_df["Max Temp"] < 80) & (City_Data_df["Max Temp"] > 70) \
                                    & (City_Data_df["Wind Speed"] < 10) \
                                    & (City_Data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,aksu,41.12,80.26,78.69,17,0,4.72,CN,1590244150
121,121,grand gaube,-20.01,57.66,73.00,69,0,5.01,MU,1590244160
152,152,hermanus,-34.42,19.23,72.00,59,0,1.01,ZA,1590244164
243,243,caconda,-13.73,15.07,77.70,29,0,3.00,AO,1590244174
257,257,sao filipe,14.90,-24.50,76.32,67,0,6.42,CV,1590244176
272,272,lubango,-14.92,13.49,75.31,31,0,4.90,AO,1590243923
286,286,leiyang,26.40,112.86,75.70,77,0,0.89,CN,1590244179
291,291,bubaque,11.28,-15.83,79.29,71,0,9.28,GW,1590244180
321,321,cap malheureux,-19.98,57.61,73.00,69,0,5.01,MU,1590244184
375,375,pangai,-19.80,-174.35,76.42,70,0,9.37,TO,1590244189


In [6]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
45,aksu,CN,41.12,80.26,
121,grand gaube,MU,-20.01,57.66,
152,hermanus,ZA,-34.42,19.23,
243,caconda,AO,-13.73,15.07,
257,sao filipe,CV,14.90,-24.50,
272,lubango,AO,-14.92,13.49,
286,leiyang,CN,26.40,112.86,
291,bubaque,GW,11.28,-15.83,
321,cap malheureux,MU,-19.98,57.61,
375,pangai,TO,-19.80,-174.35,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": Google_API_Key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
45,aksu,CN,41.12,80.26,
121,grand gaube,MU,-20.01,57.66,
152,hermanus,ZA,-34.42,19.23,
243,caconda,AO,-13.73,15.07,
257,sao filipe,CV,14.90,-24.50,
272,lubango,AO,-14.92,13.49,
286,leiyang,CN,26.40,112.86,
291,bubaque,GW,11.28,-15.83,
321,cap malheureux,MU,-19.98,57.61,
375,pangai,TO,-19.80,-174.35,


In [8]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))